使用judge数据集预训练，在使用addition数据集fine-tune之后，在模型采样阶段mask掉除了T和F外的所有字符，进行采样

In [1]:
# 重写GPT class
from model import GPT, GPTConfig
from main_utils import *

meta_path = 'meta_all_ascii_chars.pkl'
encode, decode = get_encode_decode(meta_path)

Loading meta from meta_all_ascii_chars.pkl...


In [2]:
encode('TF')

[53, 39]

T和F的编码分别是53和39，我们只需要gpt model在遇到字符'~'之后生成下一个字符时mask掉除了53和39以外的位置即可

In [3]:
ckpt_path = 'out/out-addition-from-judge/ckpt_acc.pt'
mydevice = 'cuda'
checkpoint = torch.load(ckpt_path, map_location=mydevice)
gptconf = GPTConfig(**checkpoint['model_args'])
model = GPT(gptconf)
state_dict = checkpoint['model']
unwanted_prefix = '_orig_mod.'
for k,v in list(state_dict.items()):
    if k.startswith(unwanted_prefix):
        state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
model.load_state_dict(state_dict)
model.to(mydevice)

number of parameters: 10.66M


GPT(
  (transformer): ModuleDict(
    (wte): Embedding(96, 384)
    (wpe): Embedding(256, 384)
    (drop): Dropout(p=0.2, inplace=False)
    (h): ModuleList(
      (0-5): 6 x Block(
        (ln_1): LayerNorm()
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=384, out_features=1152, bias=False)
          (c_proj): Linear(in_features=384, out_features=384, bias=False)
          (attn_dropout): Dropout(p=0.2, inplace=False)
          (resid_dropout): Dropout(p=0.2, inplace=False)
        )
        (ln_2): LayerNorm()
        (mlp): MLP(
          (c_fc): Linear(in_features=384, out_features=1536, bias=False)
          (gelu): GELU(approximate='none')
          (c_proj): Linear(in_features=1536, out_features=384, bias=False)
          (dropout): Dropout(p=0.2, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm()
  )
  (lm_head): Linear(in_features=384, out_features=96, bias=False)
)

In [4]:
input_exp = 'j(1+1=2)~'
start_ids = encode(input_exp)
# 将编码转换为张量，并额外加一个维度，从len(start_ids)变为(1,len(start_ids))
x = (torch.tensor(start_ids, dtype=torch.long, device=mydevice)[None, ...])
model.generate(x, max_new_tokens=1)

tensor([[75,  9, 18, 12, 18, 30, 19, 10, 95, 27]], device='cuda:0')

In [5]:
start_ids

[75, 9, 18, 12, 18, 30, 19, 10, 95]

In [6]:
decode([18])

'1'

下面重新写一个gpt类，继承原来的类，但重写generate方法

In [7]:
from torch.nn import functional as F
class JudgeGPT(GPT):
    @torch.no_grad()
    def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None):
        """
        Take a conditioning sequence of indices idx (LongTensor of shape (b,t)) and complete
        the sequence max_new_tokens times, feeding the predictions back into the model each time.
        Most likely you'll want to make sure to be in model.eval() mode of operation for this.
        """
        for _ in range(max_new_tokens):
            # if the sequence context is growing too long we must crop it at block_size
            idx_cond = idx if idx.size(1) <= self.config.block_size else idx[:, -self.config.block_size:]
            # forward the model to get the logits for the index in the sequence
            logits, _ = self(idx_cond)
            # pluck the logits at the final step and scale by desired temperature
            logits = logits[:, -1, :] / temperature
            # create a mask with -Inf for all indices except 39 and 58
            mask = torch.full_like(logits, -float('Inf'))
            mask[:, [39, 53]] = 0
            logits += mask
            # optionally crop the logits to only the top k options
            if top_k is not None:
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                logits[logits < v[:, [-1]]] = -float('Inf')
            # apply softmax to convert logits to (normalized) probabilities
            probs = F.softmax(logits, dim=-1)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)
            # append sampled index to the running sequence and continue
            idx = torch.cat((idx, idx_next), dim=1)

        return idx

In [8]:
ckpt_path = 'out/out-addition-from-judge/ckpt_acc.pt'
mydevice = 'cuda'
checkpoint = torch.load(ckpt_path, map_location=mydevice)
gptconf = GPTConfig(**checkpoint['model_args'])
model = JudgeGPT(gptconf)
state_dict = checkpoint['model']
unwanted_prefix = '_orig_mod.'
for k,v in list(state_dict.items()):
    if k.startswith(unwanted_prefix):
        state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
model.load_state_dict(state_dict)
model.to(mydevice)

number of parameters: 10.66M


JudgeGPT(
  (transformer): ModuleDict(
    (wte): Embedding(96, 384)
    (wpe): Embedding(256, 384)
    (drop): Dropout(p=0.2, inplace=False)
    (h): ModuleList(
      (0-5): 6 x Block(
        (ln_1): LayerNorm()
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=384, out_features=1152, bias=False)
          (c_proj): Linear(in_features=384, out_features=384, bias=False)
          (attn_dropout): Dropout(p=0.2, inplace=False)
          (resid_dropout): Dropout(p=0.2, inplace=False)
        )
        (ln_2): LayerNorm()
        (mlp): MLP(
          (c_fc): Linear(in_features=384, out_features=1536, bias=False)
          (gelu): GELU(approximate='none')
          (c_proj): Linear(in_features=1536, out_features=384, bias=False)
          (dropout): Dropout(p=0.2, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm()
  )
  (lm_head): Linear(in_features=384, out_features=96, bias=False)
)

In [9]:
input_exp = 'j(1+1=2)~'
start_ids = encode(input_exp)
# 将编码转换为张量，并额外加一个维度，从len(start_ids)变为(1,len(start_ids))
x = (torch.tensor(start_ids, dtype=torch.long, device=mydevice)[None, ...])
model.generate(x, max_new_tokens=1)

tensor([[75,  9, 18, 12, 18, 30, 19, 10, 95, 39]], device='cuda:0')

In [10]:
input_exp = 'j(1+1=3)~'
start_ids = encode(input_exp)
# 将编码转换为张量，并额外加一个维度，从len(start_ids)变为(1,len(start_ids))
x = (torch.tensor(start_ids, dtype=torch.long, device=mydevice)[None, ...])
model.generate(x, max_new_tokens=1)

tensor([[75,  9, 18, 12, 18, 30, 20, 10, 95, 39]], device='cuda:0')

可见model似乎还是有一定的判断能力

## 测试

使用judgeGPT,测试用pretrain-jugde+addition fine tune的model

先使用普通GPT

In [12]:
from test_utils import model_tester

device = 'cuda'
model_path = 'out/out-addition-from-judge/ckpt_acc.pt'
tester = model_tester(
    model_path=model_path,
    extra_test_file='./test_data/extra_num_judge_prompt.txt',
    add_noise_test_file='./test_data/add_noise_judge_prompt.txt',
    mode='judge_op',
    mydevice=device,
    judge_gpt=False
)
tester.test_extra()

number of parameters: 10.66M
Loading meta from meta_all_ascii_chars.pkl...
evaluating addition from: FILE:./test_data/extra_num_judge_prompt.txt


 34%|███▍      | 28/82 [00:00<00:00, 136.33it/s]

no judging outputs(x):  j(836+149=985)~(
no judging outputs(x):  j(138+435=573)~5
no judging outputs(x):  j(261+552=813)~2
no judging outputs(x):  j(358+424=782)~:
no judging outputs(x):  j(97+924=1021)~1
no judging outputs(x):  j(332+507=839)~(
no judging outputs(x):  j(635+78=7136)~(
no judging outputs(x):  j(395+39=4341)~)
no judging outputs(x):  j(862+118=980)~(
no judging outputs(x):  j(419+54=4736)~(
no judging outputs(x):  j(297+407=704)~(
no judging outputs(x):  j(641+47=8688)~6
no judging outputs(x):  j(579+135=714)~8
no judging outputs(x):  j(241+39=2804)~8
no judging outputs(x):  j(545+394=939)~1
no judging outputs(x):  j(600+151=751)~8
no judging outputs(x):  j(920+24=9446)~(
no judging outputs(x):  j(245+29=2742)~:
no judging outputs(x):  j(18+667=6857)~(
no judging outputs(x):  j(171+158=329)~5
no judging outputs(x):  j(397+336=733)~(
no judging outputs(x):  j(366+141=507)~(
no judging outputs(x):  j(516+252=768)~(
no judging outputs(x):  j(312+156=468)~6
no judging outpu

 71%|███████   | 58/82 [00:00<00:00, 140.75it/s]

no judging outputs(x):  j(578+763=13414)~6
no judging outputs(x):  j(964+266=12307)~1
no judging outputs(x):  j(354+722=91076)~(
no judging outputs(x):  j(941+928=18699)~3
no judging outputs(x):  j(580+587=11167)~2
no judging outputs(x):  j(893+940=21833)~7
no judging outputs(x):  j(979+198=71177)~3
no judging outputs(x):  j(652+431=10835)~9
no judging outputs(x):  j(698+933=16317)~1
no judging outputs(x):  j(858+177=61035)~:
no judging outputs(x):  j(184+975=11597)~(
no judging outputs(x):  j(738+459=11972)~7
no judging outputs(x):  j(222+844=10664)~6
no judging outputs(x):  j(691+817=81508)~9
no judging outputs(x):  j(740+628=13687)~:
no judging outputs(x):  j(330+866=11967)~1
no judging outputs(x):  j(419+767=11864)~9
no judging outputs(x):  j(718+651=51369)~9
no judging outputs(x):  j(703+940=16438)~4
no judging outputs(x):  j(474+991=14657)~4
no judging outputs(x):  j(338+704=10426)~3
no judging outputs(x):  j(836+312=11481)~6
no judging outputs(x):  j(492+884=13764)~7
no judging 

100%|██████████| 82/82 [00:00<00:00, 142.58it/s]

no judging outputs(x):  j(219+225=4441)~4
no judging outputs(x):  j(843+739=1582)~(
no judging outputs(x):  j(716+214=9309)~0
no judging outputs(x):  j(864+973=1837)~3
no judging outputs(x):  j(322+453=7755)~(
no judging outputs(x):  j(496+620=1116)~(
no judging outputs(x):  j(416+489=9055)~:
no judging outputs(x):  j(368+850=1218)~(
no judging outputs(x):  j(748+972=1720)~:
no judging outputs(x):  j(216+678=8948)~(
no judging outputs(x):  j(501+495=7996)~(
no judging outputs(x):  j(949+705=1654)~(
no judging outputs(x):  j(456+147=6032)~(
no judging outputs(x):  j(109+463=5728)~2
no judging outputs(x):  j(106+183=2891)~8
no judging outputs(x):  j(704+250=9548)~2
no judging outputs(x):  j(557+117=6742)~:
no judging outputs(x):  j(425+151=4576)~(
no judging outputs(x):  j(568+675=1243)~(
no judging outputs(x):  j(698+264=9626)~(
no judging outputs(x):  j(847+838=1685)~(
no judging outputs(x):  j(467+171=6381)~1
no judging outputs(x):  j(355+309=6648)~(
no judging outputs(x):  j(371+130=

In [13]:
tester.test_add_noise()

evaluating addition from: FILE:./test_data/add_noise_judge_prompt.txt


 20%|█▉        | 16/81 [00:00<00:00, 154.85it/s]

no judging outputs(x):  j(836+149=985)~(
no judging outputs(x):  j(138+435=573)~5
no judging outputs(x):  j(546+141=687)~7
no judging outputs(x):  j(333+634=967)~(
no judging outputs(x):  j(247+571=848)~7
no judging outputs(x):  j(261+552=813)~2
no judging outputs(x):  j(116+507=653)~7
no judging outputs(x):  j(358+424=782)~:
no judging outputs(x):  j(162+465=707)~(
no judging outputs(x):  j(342+158=500)~6
no judging outputs(x):  j(94+966=1060)~(
no judging outputs(x):  j(204+727=931)~6
no judging outputs(x):  j(332+507=839)~(
no judging outputs(x):  j(606+207=813)~9
no judging outputs(x):  j(641+47=1388)~(
no judging outputs(x):  j(304+597=901)~6
no judging outputs(x):  j(395+19=6414)~1
no judging outputs(x):  j(579+135=714)~8
no judging outputs(x):  j(241+39=5280)~5
no judging outputs(x):  j(600+151=751)~8
no judging outputs(x):  j(666+272=938)~(
no judging outputs(x):  j(920+24=1844)~8
no judging outputs(x):  j(18+667=1085)~:
no judging outputs(x):  j(397+336=763)~(
no judging outpu

 59%|█████▉    | 48/81 [00:00<00:00, 146.96it/s]

no judging outputs(x):  j(964+586=1550)~(
no judging outputs(x):  j(833+939=1772)~(
no judging outputs(x):  j(232+825=1057)~(
no judging outputs(x):  j(666+512=1228)~3
no judging outputs(x):  j(883+714=1601)~9
no judging outputs(x):  j(937+807=1752)~(
no judging outputs(x):  j(988+366=1354)~(
no judging outputs(x):  j(921+494=1415)~1
no judging outputs(x):  j(827+385=2212)~:
no judging outputs(x):  j(263+832=5095)~6
no judging outputs(x):  j(105+937=1102)~(
no judging outputs(x):  j(97+924=71021)~8
no judging outputs(x):  j(766+895=1668)~(
no judging outputs(x):  j(766+682=9448)~1
no judging outputs(x):  j(898+525=1424)~9
no judging outputs(x):  j(462+959=1421)~7
no judging outputs(x):  j(884+490=1376)~(
no judging outputs(x):  j(660+590=1250)~(
no judging outputs(x):  j(683+133=2816)~(
no judging outputs(x):  j(312+769=1081)~2
no judging outputs(x):  j(963+780=1753)~(
no judging outputs(x):  j(946+176=1122)~(
no judging outputs(x):  j(862+118=1060)~(
no judging outputs(x):  j(948+691=

100%|██████████| 81/81 [00:00<00:00, 152.72it/s]

no judging outputs(x):  j(641+913=2054)~:
no judging outputs(x):  j(979+108=1087)~(
no judging outputs(x):  j(404+126=2530)~(
no judging outputs(x):  j(874+356=1230)~(
no judging outputs(x):  j(979+820=1809)~6
no judging outputs(x):  j(524+577=1301)~4
no judging outputs(x):  j(376+811=1187)~1
no judging outputs(x):  j(295+854=1229)~:
no judging outputs(x):  j(623+542=1165)~3
no judging outputs(x):  j(250+946=7196)~(
no judging outputs(x):  j(299+831=1130)~:
no judging outputs(x):  j(682+331=1013)~(
no judging outputs(x):  j(887+738=1625)~(
no judging outputs(x):  j(433+844=1277)~6
no judging outputs(x):  j(872+494=1566)~(
no judging outputs(x):  j(791+433=1924)~:
no judging outputs(x):  j(900+199=1100)~1
no judging outputs(x):  j(797+575=1372)~(
no judging outputs(x):  j(880+516=1396)~(
no judging outputs(x):  j(691+871=1562)~(
no judging outputs(x):  j(661+780=3441)~4
no judging outputs(x):  j(879+543=1422)~6
no judging outputs(x):  j(943+524=1487)~4
no judging outputs(x):  j(825+960=

使用judgeGPT测试

In [14]:
device = 'cuda'
model_path = 'out/out-addition-from-judge/ckpt_acc.pt'
tester = model_tester(
    model_path=model_path,
    extra_test_file='./test_data/extra_num_judge_prompt.txt',
    add_noise_test_file='./test_data/add_noise_judge_prompt.txt',
    mode='judge_op',
    mydevice=device,
    judge_gpt=True
)
tester.test_extra()

number of parameters: 10.66M
Loading meta from meta_all_ascii_chars.pkl...
evaluating addition from: FILE:./test_data/extra_num_judge_prompt.txt


 27%|██▋       | 22/82 [00:00<00:00, 210.75it/s]

wrong outputs(x):  j(836+149=985)~F
wrong outputs(x):  j(138+435=573)~F
wrong outputs(x):  j(97+924=1021)~F
wrong outputs(x):  j(332+507=839)~F
wrong outputs(x):  j(395+39=4341)~T
wrong outputs(x):  j(862+118=980)~F
wrong outputs(x):  j(920+24=9446)~T
wrong outputs(x):  j(245+29=2742)~T
wrong outputs(x):  j(18+667=6857)~T
wrong outputs(x):  j(397+336=733)~F
wrong outputs(x):  j(516+252=768)~F
wrong outputs(x):  j(50+207=2574)~T
wrong outputs(x):  j(442+351=793)~F
wrong outputs(x):  j(256+301=557)~F
wrong outputs(x):  j(219+502=721)~F
wrong outputs(x):  j(347+250=597)~F
wrong outputs(x):  j(835+160=995)~F
wrong outputs(x):  j(319+158=477)~F
wrong outputs(x):  j(274+384=658)~F
wrong outputs(x):  j(49+570=3619)~T
wrong outputs(x):  j(27+430=3457)~T
wrong outputs(x):  j(319+148=467)~F
wrong outputs(x):  j(389+129=518)~F
wrong outputs(x):  j(134+289=423)~F
wrong outputs(x):  j(80+702=7824)~T
wrong outputs(x):  j(308+137=445)~F
wrong outputs(x):  j(125+548=673)~F
wrong outputs(x):  j(801+27=

 78%|███████▊  | 64/82 [00:00<00:00, 189.45it/s]

wrong outputs(x):  j(576+810=13865)~T
wrong outputs(x):  j(777+783=15604)~T
wrong outputs(x):  j(851+859=71710)~T
wrong outputs(x):  j(711+696=14074)~T
wrong outputs(x):  j(652+411=10631)~T
wrong outputs(x):  j(289+711=10001)~T
wrong outputs(x):  j(707+988=61695)~T
wrong outputs(x):  j(932+408=13406)~T
wrong outputs(x):  j(874+677=15512)~T
wrong outputs(x):  j(925+361=12865)~T
wrong outputs(x):  j(866+606=91472)~T
wrong outputs(x):  j(673+463=11365)~T
wrong outputs(x):  j(820+222=10424)~T
wrong outputs(x):  j(542+559=31101)~T
wrong outputs(x):  j(633+767=14002)~T
wrong outputs(x):  j(786+373=11159)~T
wrong outputs(x):  j(904+353=12573)~T
wrong outputs(x):  j(979+820=17999)~T
wrong outputs(x):  j(295+854=11491)~T
wrong outputs(x):  j(250+946=41196)~T
wrong outputs(x):  j(299+831=21130)~T
wrong outputs(x):  j(682+331=10137)~T
wrong outputs(x):  j(837+391=12287)~T
wrong outputs(x):  j(825+960=17851)~T
wrong outputs(x):  j(600+454=61054)~T
wrong outputs(x):  j(478+620=10983)~T
wrong output

100%|██████████| 82/82 [00:00<00:00, 185.36it/s]

wrong outputs(x):  j(659+771=1430)~F
wrong outputs(x):  j(338+982=1320)~F
wrong outputs(x):  j(977+72=10496)~T
wrong outputs(x):  j(252+684=9364)~T
wrong outputs(x):  j(934+203=1137)~F
wrong outputs(x):  j(391+901=1292)~F
wrong outputs(x):  j(218+678=7896)~T
wrong outputs(x):  j(184+243=5427)~T
wrong outputs(x):  j(226+924=1150)~F
wrong outputs(x):  j(399+640=1039)~F
wrong outputs(x):  j(731+422=1153)~F
wrong outputs(x):  j(204+534=7384)~T
wrong outputs(x):  j(298+499=5797)~T
wrong outputs(x):  j(312+740=1052)~F
wrong outputs(x):  j(674+492=1166)~F
wrong outputs(x):  j(658+336=9944)~T
wrong outputs(x):  j(720+891=1611)~F
wrong outputs(x):  j(820+271=1091)~F
wrong outputs(x):  j(659+762=1421)~F
wrong outputs(x):  j(317+850=1167)~F
wrong outputs(x):  j(781+188=9693)~T
wrong outputs(x):  j(733+829=1562)~F
wrong outputs(x):  j(184+147=3318)~T
wrong outputs(x):  j(120+118=2384)~T
wrong outputs(x):  j(500+884=1384)~F
wrong outputs(x):  j(670+153=8236)~T
wrong outputs(x):  j(351+212=5632)~T
w

In [15]:
tester.test_add_noise()

evaluating addition from: FILE:./test_data/add_noise_judge_prompt.txt


 37%|███▋      | 30/81 [00:00<00:00, 149.38it/s]

wrong outputs(x):  j(138+435=573)~F
wrong outputs(x):  j(333+634=967)~F
wrong outputs(x):  j(94+966=1060)~F
wrong outputs(x):  j(204+727=931)~F
wrong outputs(x):  j(332+507=839)~F
wrong outputs(x):  j(641+47=1388)~T
wrong outputs(x):  j(304+597=901)~F
wrong outputs(x):  j(395+19=6414)~T
wrong outputs(x):  j(241+39=5280)~T
wrong outputs(x):  j(600+151=751)~F
wrong outputs(x):  j(920+24=1844)~T
wrong outputs(x):  j(323+141=464)~F
wrong outputs(x):  j(516+252=768)~F
wrong outputs(x):  j(32+621=5653)~T
wrong outputs(x):  j(256+301=563)~T
wrong outputs(x):  j(319+158=547)~T
wrong outputs(x):  j(274+384=658)~F
wrong outputs(x):  j(13+655=9668)~T
wrong outputs(x):  j(128+133=321)~T
wrong outputs(x):  j(519+364=913)~T
wrong outputs(x):  j(346+114=490)~T
wrong outputs(x):  j(549+160=709)~F
wrong outputs(x):  j(288+166=454)~F
wrong outputs(x):  j(110+519=629)~F
wrong outputs(x):  j(124+449=577)~T
wrong outputs(x):  j(551+389=940)~F
wrong outputs(x):  j(281+598=879)~F
wrong outputs(x):  j(125+548

 75%|███████▌  | 61/81 [00:00<00:00, 149.03it/s]

wrong outputs(x):  j(397+77=475)~T
wrong outputs(x):  j(656+73=729)~F
wrong outputs(x):  j(569+34=608)~T
wrong outputs(x):  j(241+73=314)~F
wrong outputs(x):  j(170+12=182)~F
wrong outputs(x):  j(42+810=852)~F
wrong outputs(x):  j(732+96=878)~T
wrong outputs(x):  j(952+17=969)~F
wrong outputs(x):  j(798+49=887)~T
wrong outputs(x):  j(99+646=745)~F
wrong outputs(x):  j(54+512=666)~T
wrong outputs(x):  j(23+473=502)~T
wrong outputs(x):  j(52+57=5109)~T
wrong outputs(x):  j(263+36=299)~F
wrong outputs(x):  j(64+190=254)~F
wrong outputs(x):  j(514+81=595)~F
wrong outputs(x):  j(892+11=903)~F
wrong outputs(x):  j(31+521=592)~T
wrong outputs(x):  j(74+870=948)~T
wrong outputs(x):  j(180+20=200)~F
wrong outputs(x):  j(79+856=939)~T
wrong outputs(x):  j(142+76=218)~F
wrong outputs(x):  j(69+434=503)~F
wrong outputs(x):  j(788+77=865)~F
wrong outputs(x):  j(140+14=154)~F
wrong outputs(x):  j(39+864=903)~F
wrong outputs(x):  j(246+90=336)~F
wrong outputs(x):  j(133+13=146)~F
wrong outputs(x):  j

100%|██████████| 81/81 [00:00<00:00, 150.56it/s]

wrong outputs(x):  j(957+962=1919)~F
wrong outputs(x):  j(717+569=9286)~T
wrong outputs(x):  j(669+451=1190)~T
wrong outputs(x):  j(961+247=1208)~F
wrong outputs(x):  j(174+950=1125)~T
wrong outputs(x):  j(986+773=1779)~T
wrong outputs(x):  j(698+677=1405)~T
wrong outputs(x):  j(415+882=1303)~T
wrong outputs(x):  j(862+259=1121)~F
wrong outputs(x):  j(343+965=1313)~T
wrong outputs(x):  j(261+792=1133)~T
wrong outputs(x):  j(468+713=4181)~T
wrong outputs(x):  j(321+700=6021)~T
wrong outputs(x):  j(476+973=1449)~F
wrong outputs(x):  j(803+853=8656)~T
wrong outputs(x):  j(474+931=1405)~F
wrong outputs(x):  j(357+692=1053)~T
wrong outputs(x):  j(984+232=1216)~F
wrong outputs(x):  j(352+406=5758)~T
wrong outputs(x):  j(790+483=1273)~F
wrong outputs(x):  j(946+153=1199)~T
wrong outputs(x):  j(568+591=1189)~T
wrong outputs(x):  j(957+729=1686)~F
wrong outputs(x):  j(886+456=1392)~T
wrong outputs(x):  j(647+826=1473)~F
wrong outputs(x):  j(217+513=2730)~T
wrong outputs(x):  j(259+161=7420)~T
w

准确率仅50%左右，基本是纯瞎猜的水平